In [27]:
import re
lista_strings = ["Olá, mundo!", "Este é um exemplo de string com @caracteres especiais#"]


def limpar_caracteres_especiais(string):

    return re.sub(r'[^\w\s]', '', string)


lista_limpa = [limpar_caracteres_especiais(string) for string in lista_strings]


for string in lista_limpa:
    print(string)

Olá mundo
Este é um exemplo de string com caracteres especiais


In [28]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [29]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\guido\OneDrive\Documentos\GitHub\Cdados


In [30]:
train = pd.read_excel('dados/dados_treino.xlsx')
train.head(5)

,Target,Mensagem
0,Positivo,"Leitura rápida, com algumas dicas para expandi..."
1,Neutro,"Coaching virou um grande modismo, assim como c..."
2,Negativo,O fundo da história é pra se pensar. Mas é mui...
3,Neutro,A tradução é péssima e diferente do original. ...
4,Negativo,Achei sem graça esse livro. Tópicos de livros ...


In [31]:
test = pd.read_excel('dados/dados_teste.xlsx')
test.head(5)

,Target,Mensagem
0,Neutro,"É incrível como os sebos não tem noção, vendem..."
1,Negativo,Livro sem nenhuma coisa de diferente. Autor ba...
2,Negativo,Bem fraco. Achei muito forçada a ligação com o...
3,Neutro,A caixa veio rasgada. como fazia mais de sete ...
4,Negativo,"Que livro chato, gente quase não consegui term..."


In [63]:
df = pd.read_excel('dados/dados_treino.xlsx')

coluna_especifica = df['Mensagem']



def limpar_caracteres_especiais(string):

    return re.sub(r'[^\w\s]', '', string)


nova_coluna_especifica = [limpar_caracteres_especiais(string) for string in coluna_especifica]
nova_coluna_especifica=[string.lower() for string in nova_coluna_especifica]
train.Mensagem=nova_coluna_especifica
train.Mensagem.to_frame()
train

,Target,Mensagem
0,Positivo,leitura rápida com algumas dicas para expandir...
1,Neutro,coaching virou um grande modismo assim como cr...
2,Negativo,o fundo da história é pra se pensar mas é muit...
3,Neutro,a tradução é péssima e diferente do original a...
4,Negativo,achei sem graça esse livro tópicos de livros a...
...,...,...
745,Negativo,conteúdo superficial onde se propõe nas dicas ...
746,Negativo,péssimo
747,Negativo,autor confunde tecnicas de coaching com religi...
748,Negativo,livro superficial escrito para impressionar os...


In [61]:
palavras=[]
coluna_especifica = train.Mensagem
for frase in coluna_especifica:
    palavra=frase.split()
    for e in palavra:
        palavras.append(e)
palavras

['leitura',
 'rápida',
 'com',
 'algumas',
 'dicas',
 'para',
 'expandirmos',
 'nossa',
 'percepção',
 'sobre',
 'nossas',
 'ideias',
 'ou',
 'ações',
 'acredito',
 'que',
 'no',
 'final',
 'tenha',
 'agregado',
 'algo',
 'mesmo',
 'que',
 'seja',
 'mínimo',
 'coaching',
 'virou',
 'um',
 'grande',
 'modismo',
 'assim',
 'como',
 'criticálo',
 'uma',
 'infeliz',
 'realidade',
 'preciso',
 'assumir',
 'como',
 'um',
 'profundo',
 'amante',
 'do',
 'autodesenvolvimento',
 'consegui',
 'obter',
 'diversos',
 'insights',
 'úteis',
 'durante',
 'a',
 'leitura',
 'ou',
 'seja',
 'de',
 'qualquer',
 'forma',
 'dá',
 'pra',
 'extrair',
 'boas',
 'ideais',
 'mas',
 'como',
 'de',
 'praxe',
 'no',
 'mundo',
 'do',
 'coaching',
 'as',
 'páginas',
 'começam',
 'a',
 'se',
 'encher',
 'de',
 'mensagens',
 'e',
 'afirmações',
 'sem',
 'embasamento',
 'nenhum',
 'uma',
 'verdadeira',
 'compilação',
 'de',
 'achismo',
 'e',
 'falsas',
 'verdades',
 'baseadas',
 'em',
 'princípios',
 'infudados',
 'tir